In [ ]:
# default_exp loss

# Losses

> Loss callback functions

In [ ]:
#hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
# export

from mrl.imports import *
from mrl.torch_imports import *
from mrl.torch_core import *
from mrl.callbacks import *
from mrl.policy_gradient import *

In [ ]:
# export

class LossCallback(Callback):
    def __init__(self, loss_function, name, weight=1., track=True, order=20):
        super().__init__(name=name, order=order)
        self.loss_function = loss_function
        self.weight = weight
        self.track = track
        
    def setup(self):
        if self.track:
            log = self.environment.log
            log.add_metric(self.name)
            log.add_log(self.name)
        
    def compute_loss(self):
        loss, loss_dict = self.loss_function.from_batch_state(self.batch_state)
        
        if self.track:
            self.environment.log.update_metric(self.name, loss.mean().detach().cpu().numpy())
            
        loss = loss * self.weight
        self.environment.batch_state.loss += loss.mean()
        self.environment.batch_state[self.name] = loss.detach().cpu().numpy()

In [ ]:
class PolicyLoss(LossCallback):
    def __init__(self, policy_function, name, value_head=None, 
                 v_update=0.95, v_update_iter=10,
                 vopt_kwargs={}, weight=1, track=True):
        assert isinstance(policy_function, BasePolicy)
        super().__init__(policy_function, name, weight, track)
        
        self.set_model(value_head, vopt_kwargs)
        self.v_update = v_update
        self.v_update_iter = v_update_iter
        self.fields = [
            'model_gathered_logprobs',
            'base_gathered_logprobs',
            'mask',
            'trajectory_rewards',
            'model_logprobs',
            'base_logprobs',
            'value_input'
        ]
        
    def set_model(self, value_head, vopt_kwargs):
        self.value_head = value_head
        if self.value_head is not None:
            self.base_value_head = copy.deepcopy(self.value_head)
            to_device(self.value_head)
            to_device(self.base_value_head)
            self.opt = optim.Adam(self.value_head.parameters(), **vopt_kwargs)
            
    def before_batch(self):
        env = self.environment
        batch_state = env.batch_state
        for field in self.fields:
            if not hasattr(batch_state, field):
                batch_state[field] = None
                
    def get_model_outputs(self):
            
        env = self.environment
        batch_state = env.batch_state
        value_input = batch_state.value_input
        
        if (self.value_head is not None) and (value_input is not None):
            value_predictions = self.agent.value_head(value_input)
            with torch.no_grad():
                base_value_predictions = self.base_value_head(value_input)
                
        else:
            value_predictions = None
            base_value_predictions = None
            
        batch_state.state_values = value_predictions
        batch_state.ref_state_values = base_value_predictions
            
    def zero_grad(self):
        self.opt.zero_grad()
        
    def step(self):
        self.opt.step()
        
    def after_batch(self):
        log = self.environment.log
        iterations = log.iterations
        if iterations%self.v_update_iter == 0 and iterations>0:
            self.update_base_model()
            
    def update_base_model(self):
        if self.value_head is not None:
            if self.v_update < 1:
                merge_models(self.base_value_head, self.value_head, alpha=self.v_update)
                
    def save_weights(self, filename):
        state_dict = {}
            
        if isinstance(self.value_head, nn.Module):
            state_dict['value_head'] = self.value_head.state_dict()
            state_dict['base_value_head'] = self.base_value_head.state_dict()
        else:
            state_dict['value_head'] = None
            state_dict['base_value_head'] = None
            
        torch.save(state_dict, filename)
        
    def load_weights(self, filename):
        state_dict = torch.load(filename, map_location=get_model_device(self.model))
            
        if isinstance(self.value_head, nn.Module):
            self.value_head.load_state_dict(state_dict['value_head'])
            self.base_value_head.load_state_dict(state_dict['base_value_head'])
